In [1]:
from datetime import datetime, timezone
from urllib.parse import urlencode
import webbrowser

import requests
from dateutil.parser import parse as parse_date
from funcy import post_processing, project
from ipyleaflet import Map, Marker, MarkerCluster


lat, lng = 51.525609042041864, -0.12358503246188147
qs = {
    'include[0]': 'theaters',
    'include[1]': 'exhibitors',
    'include[2]': 'movies',
    'include[3]': 'formats',
    'filter[titles][slug][0]': 'apollo-11',
    'filter[theaters][lat]': lat,
    'filter[theaters][lon]': lng,
    'filter[theaters][rf]': '3',
    'filter[theaters][country][0]': 'gu',
    'filter[theaters][country][1]': 'gg',
    'filter[theaters][country][2]': 'im',
    'filter[theaters][country][3]': 'gb',
    'filter[theaters][country][4]': 'je',
    'filter[theaters][country][5]': 'ie',
}

url = f'https://stdata.powster.com/screenings?{urlencode(qs)}'

In [2]:
response = requests.get(url)
response.raise_for_status()
doc = response.json()

In [3]:
for key, value in doc.items():
    print(key, len(value))

data 453
meta 0
included 154


In [4]:
@post_processing(list)
def get_screenings(data):
    seen = set()
    for screening in data:
        s = screening['attributes']
        if not s['date'] or not s['time']:
            continue
        time = parse_date(s['time'])
        date = parse_date(s['date'])
        dt = datetime.combine(date, time.time())
        s['datetime'] = dt.replace(tzinfo=dt.tzinfo or timezone.utc)
        unique_data = project(s, ('datetime', 'theater')).__str__()
        if unique_data in seen:
            continue
        seen.add(unique_data)
        yield s
    
screenings = get_screenings(doc['data'])
project(screenings[0], ('date', 'time', 'datetime'))

{'date': '2019-07-25T00:00:00.000Z',
 'time': '16:00:00',
 'datetime': datetime.datetime(2019, 7, 25, 16, 0, tzinfo=datetime.timezone.utc)}

In [5]:
@post_processing(list)
def filter_by_date(screenings, start, end):
    for s in screenings:
        if start <= s['datetime'] <= end:
            yield s

start = datetime(2019, 7, 22, 17, tzinfo=timezone.utc)
end = datetime(2019, 7, 22, 22, tzinfo=timezone.utc)
dated = filter_by_date(screenings, start, end)
dated[0]

{'movie_id': 'e6a02464-7576-11e9-bd37-ab00ceaad779',
 'theater_id': 'da6f4a38-d738-11e5-bccb-937c20d316a2',
 'provider_id': 'pow',
 'provider_screening_id': None,
 'date': '2019-07-22T00:00:00.000Z',
 'time': '20:30:00',
 'url': 'https://www.jw3.org.uk/event/apollo-11#.XSSh-OgzZPY',
 'cta': None,
 'is_visible': True,
 'tz_offset': None,
 'doc': None,
 'datetime': datetime.datetime(2019, 7, 22, 20, 30, tzinfo=datetime.timezone.utc)}

In [6]:
doc['included'][0]

{'type': 'movies',
 'id': '06784d4e-96aa-11e9-a442-03a76d585de9',
 'attributes': {'provider_id': 'mx',
  'provider_movie_id': '2b83f8b7-4fb3-4f91-8699-f6217e321610',
  'format_id': '4c5e546c-2d46-11e8-9156-9f1ab0ac9b09',
  'title': 'Apollo 11',
  'official_site': None,
  'fandango_movie_id': None,
  'movietickets_movie_id': None,
  'movie_master_id': '91c2ec6c-1b6a-11e9-9052-43415f526ce1',
  'release_date': None,
  'meta': {'cast': ['Michael Collins', 'Neil Armstrong', 'Buzz Aldrin'],
   'genre': ['documentary'],
   'director': ['Todd Douglas Miller']},
  'slug': 'apollo-11'}}

In [7]:
theaters = {item['id']: item['attributes'] for item in doc['included'] if item['type'] == 'theaters'}
next(iter(theaters.values()))

{'theater_master_id': '2eb763be-5701-11e9-a90f-ab6c3fa81ab1',
 'name': 'Archlight Cinemas',
 'address': 'Arches Lane',
 'city': 'London',
 'postcode': 'SW11 8AB',
 'country': 'gb',
 'state': 'UK',
 'url': 'http://archlightcinemas.co.uk',
 'name_en': None,
 'address_en': None,
 'city_en': None,
 'lat': 51.48313,
 'lon': -0.14911,
 'tz': None,
 'tmsid': None,
 'tms_id': None,
 'fandango_theater_id': None,
 'movietickets_theater_id': None,
 'atom_theater_id': None,
 'wwm_ticketing': False,
 'wwm_tickettype': None,
 'exhibitor_id': '03f71480-0e59-11e9-86d2-d7cb380d92c4',
 'provider_id': 'wwm',
 'provider_theater_id': '48735',
 'provider_exhibitor_id': None}

In [8]:
@post_processing(list)
def include_theaters(screenings):
    for s in screenings:
        theater_id = s.get('theater_id')
        if not theater_id:
            continue
        theater = theaters[theater_id]
        s['theater'] = theater
        s['map_title'] = f'{s["time"]} {s["theater"]["name"]}'
        yield s

theater_screenings = include_theaters(dated)
theater_screenings[0]['datetime'], theater_screenings[0]['theater']

(datetime.datetime(2019, 7, 22, 20, 30, tzinfo=datetime.timezone.utc),
 {'theater_master_id': '7dd3e64c-f026-11e5-85d5-33333048279d',
  'name': 'JW3',
  'address': '341-351 Finchley Road',
  'city': 'London',
  'postcode': 'NW3 6ET',
  'country': 'gb',
  'state': None,
  'url': 'http://www.jw3.org.uk/cinema',
  'name_en': None,
  'address_en': None,
  'city_en': None,
  'lat': 51.551052,
  'lon': -0.184781,
  'tz': None,
  'tmsid': None,
  'tms_id': None,
  'fandango_theater_id': None,
  'movietickets_theater_id': None,
  'atom_theater_id': None,
  'wwm_ticketing': None,
  'wwm_tickettype': None,
  'exhibitor_id': 'e8fa8460-cfe9-11e5-b5e5-d78be98fcd64',
  'provider_id': 'pow',
  'provider_theater_id': 'jw3|london|nw36et',
  'provider_exhibitor_id': None})

In [9]:
time_to_title = [s['map_title'] for s in theater_screenings]
for s in sorted(time_to_title):
    print(s)

18:20:00 Crouch End Picturehouse
18:30:00 Bromley Picturehouse
19:15:00 Vue - Finchley Road / 02 Centre
19:40:00 Archlight Cinemas
20:00:00 Cineworld Cinema - Fulham Road
20:30:00 JW3
20:35:00 BFI Southbank
20:40:00 Curzon Bloomsbury
21:35:00 Picturehouse Central


In [10]:
@post_processing(list)
def get_markers(screenings):
    coords_to_url = {}
    for s in screenings:
        theater = s.get('theater')
        if not theater:
            continue
        lat, lng = theater.get('lat'), theater.get('lon')
        if lat is None or lng is None:
            continue
            
        coords_to_url[(lat, lng)] = s['url']
        title = s['map_title']
        mkr = Marker(location=(lat, lng), draggable=False, title=title)
        
        def handle_click(*args, **kwargs):
            url = coords_to_url[tuple(kwargs['coordinates'])]
            webbrowser.open(url)
    
        mkr.on_click(handle_click)
        
        yield mkr

m = Map(center=(lat, lng), zoom=13)
marker_cluster = MarkerCluster(markers=get_markers(theater_screenings))
m.add_layer(marker_cluster)
m.layout.height = '600px'
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …